
- Repeat for different actions - e.i. changing percent to sell or buy all
- Train MC and SARSA 
    - Compare which one performs better in training
    - Compare which one performs better on Test time period


- Try negative rewards
    - Repeat for different actions - e.i. changing percent to sell or buy all
    - Train MC and SARSA 
        - Compare which one performs better in training
        - Compare which one performs better on Test time period

- Plot percent of state space experienced
- Plot actions that resulted in Gain - may only be relevant for 100%  buy sell version


